In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
  df = fetch_datasets()['oil']
  X, y = df.data, df.target
  le = LabelEncoder()
  y = le.fit_transform(y)
  G = len(np.unique(y))
  print(np.shape(X), G, sum(y==0)*.6*.5, sum(y==1)*.6*.5)
  X[:1]

(937, 49) 2 268.8 12.299999999999999


array([[ 1.000000e+00,  2.558000e+03,  1.506090e+03,  4.566300e+02,
         9.000000e+01,  6.395000e+06,  4.088000e+01,  7.890000e+00,
         2.978000e+04,  1.900000e-01,  2.147000e+02,  2.100000e-01,
         2.600000e-01,  4.900000e-01,  1.000000e-01,  4.000000e-01,
         9.959000e+01,  3.219000e+01,  1.840000e+00,  1.600000e-01,
         2.000000e-01,  8.765000e+01,  0.000000e+00,  4.700000e-01,
         1.327800e+02, -1.000000e-02,  3.780000e+00,  2.200000e-01,
         3.200000e+00, -3.710000e+00, -1.800000e-01,  2.190000e+00,
         0.000000e+00,  2.190000e+00,  3.100000e+02,  1.611000e+04,
         0.000000e+00,  1.386800e+02,  8.900000e+01,  6.900000e+01,
         2.850000e+03,  1.000000e+03,  7.631600e+02,  1.354600e+02,
         3.730000e+00,  0.000000e+00,  3.324319e+04,  6.574000e+01,
         7.950000e+00]])

In [10]:
missing_rates = np.array([0.0, 0.1, 0.2, 0.3, 0.4])
res_all = []
n = np.array([-200,100])
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [11]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [12]:
leftindex = pd.MultiIndex.from_product([["0%", "10%", "20%", "30%", "40%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.491$\pm$0.053   0.959$\pm$0.01   
                         LR          0.539$\pm$0.081  0.951$\pm$0.009   
                         DCT         0.512$\pm$0.138  0.946$\pm$0.009   
             sensitivity SVM         0.369$\pm$0.051   0.962$\pm$0.01   
                         LR          0.415$\pm$0.083    0.95$\pm$0.01   
                         DCT           0.4$\pm$0.143  0.948$\pm$0.011   
10%          F-1 score   SVM         0.496$\pm$0.051  0.958$\pm$0.009   
                         LR          0.651$\pm$0.133  0.953$\pm$0.005   
                         DCT           0.65$\pm$0.16  0.942$\pm$0.016   
             sensitivity SVM         0.377$\pm$0.045  0.962$\pm$0.009   
                         LR           0.547$\pm$0.15  0.953$\pm$0.004   
                         DCT         0.551$\pm$0.176  0.943$\pm$0.018   
20%          F-1 score   SVM         0.469$\pm$0.037  0.964$\pm$0.011   
                         LR          0.487$\pm$0.048   0.948$\pm$0.01   
                         DCT         0.508$\pm$0.156   0.95$\pm$0.009   
             sensitivity SVM         0.346$\pm$0.028   0.966$\pm$0.01   
                         LR          0.363$\pm$0.038  0.942$\pm$0.012   
                         DCT         0.398$\pm$0.159  0.947$\pm$0.009   
30%          F-1 score   SVM         0.521$\pm$0.084  0.961$\pm$0.011   
                         LR          0.574$\pm$0.126  0.951$\pm$0.005   
                         DCT         0.523$\pm$0.166  0.936$\pm$0.014   
             sensitivity SVM         0.398$\pm$0.086   0.966$\pm$0.01   
                         LR          0.457$\pm$0.148  0.947$\pm$0.007   
                         DCT         0.414$\pm$0.165  0.935$\pm$0.014   
40%          F-1 score   SVM         0.566$\pm$0.147  0.949$\pm$0.012   
                         LR           0.604$\pm$0.16   0.94$\pm$0.011   
                         DCT         0.526$\pm$0.102  0.934$\pm$0.011   
             sensitivity SVM         0.457$\pm$0.154  0.954$\pm$0.011   
                         LR          0.502$\pm$0.179  0.933$\pm$0.014   
                         DCT           0.411$\pm$0.1  0.933$\pm$0.014   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.957$\pm$0.009   0.96$\pm$0.007   
                         LR           0.945$\pm$0.01  0.955$\pm$0.009   
                         DCT         0.942$\pm$0.007   0.941$\pm$0.01   
             sensitivity SVM          0.958$\pm$0.01  0.965$\pm$0.005   
                         LR          0.942$\pm$0.012  0.957$\pm$0.009   
                         DCT          0.938$\pm$0.01  0.941$\pm$0.013   
10%          F-1 score   SVM         0.952$\pm$0.004  0.959$\pm$0.006   
                         LR          0.944$\pm$0.008  0.953$\pm$0.006   
                         DCT          0.934$\pm$0.01  0.941$\pm$0.006   
             sensitivity SVM         0.951$\pm$0.006  0.964$\pm$0.004   
                         LR          0.939$\pm$0.012  0.954$\pm$0.007   
                         DCT         0.931$\pm$0.013   0.94$\pm$0.008   
20%          F-1 score   SVM         0.959$\pm$0.009  0.967$\pm$0.011   
                         LR          0.939$\pm$0.006  0.954$\pm$0.012   
                         DCT         0.943$\pm$0.016  0.939$\pm$0.012   
             sensitivity SVM         0.957$\pm$0.009   0.97$\pm$0.008   
                         LR          0.927$\pm$0.007   0.95$\pm$0.012   
                         DCT          0.936$\pm$0.02  0.935$\pm$0.011   
30%          F-1 score   SVM          0.959$\pm$0.01   0.962$\pm$0.01   
                         LR          0.942$\pm$0.011  0.958$\pm$0.009   
                         DCT          0.939$\pm$0.01   0.94$\pm$0.013   
             sensitivity SVM

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.491$\pm$0.053   0.959$\pm$0.01   
                         LR          0.539$\pm$0.081  0.951$\pm$0.009   
                         DCT         0.512$\pm$0.138  0.946$\pm$0.009   
             sensitivity SVM         0.369$\pm$0.051   0.962$\pm$0.01   
                         LR          0.415$\pm$0.083    0.95$\pm$0.01   
                         DCT           0.4$\pm$0.143  0.948$\pm$0.011   
10%          F-1 score   SVM         0.496$\pm$0.051  0.958$\pm$0.009   
                         LR          0.651$\pm$0.133  0.953$\pm$0.005   
                         DCT           0.65$\pm$0.16  0.942$\pm$0.016   
             sensitivity SVM         0.377$\pm$0.045  0.962$\pm$0.009   
                         LR           0.547$\pm$0.15  0.953$\pm$0.004   
                         DCT         0.551$\pm$0.176  0.943$\pm$0.018   
20%          F-1 score   SVM         0.469$\pm$0.037  0.964$\pm$0.011   
                         LR          0.487$\pm$0.048   0.948$\pm$0.01   
                         DCT         0.508$\pm$0.156   0.95$\pm$0.009   
             sensitivity SVM         0.346$\pm$0.028   0.966$\pm$0.01   
                         LR          0.363$\pm$0.038  0.942$\pm$0.012   
                         DCT         0.398$\pm$0.159  0.947$\pm$0.009   
30%          F-1 score   SVM         0.521$\pm$0.084  0.961$\pm$0.011   
                         LR          0.574$\pm$0.126  0.951$\pm$0.005   
                         DCT         0.523$\pm$0.166  0.936$\pm$0.014   
             sensitivity SVM         0.398$\pm$0.086   0.966$\pm$0.01   
                         LR          0.457$\pm$0.148  0.947$\pm$0.007   
                         DCT         0.414$\pm$0.165  0.935$\pm$0.014   
40%          F-1 score   SVM         0.566$\pm$0.147  0.949$\pm$0.012   
                         LR           0.604$\pm$0.16   0.94$\pm$0.011   
                         DCT         0.526$\pm$0.102  0.934$\pm$0.011   
             sensitivity SVM         0.457$\pm$0.154  0.954$\pm$0.011   
                         LR          0.502$\pm$0.179  0.933$\pm$0.014   
                         DCT           0.411$\pm$0.1  0.933$\pm$0.014   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.957$\pm$0.009   0.96$\pm$0.007   
                         LR           0.945$\pm$0.01  0.955$\pm$0.009   
                         DCT         0.942$\pm$0.007   0.941$\pm$0.01   
             sensitivity SVM          0.958$\pm$0.01  0.965$\pm$0.005   
                         LR          0.942$\pm$0.012  0.957$\pm$0.009   
                         DCT          0.938$\pm$0.01  0.941$\pm$0.013   
10%          F-1 score   SVM         0.952$\pm$0.004  0.959$\pm$0.006   
                         LR          0.944$\pm$0.008  0.953$\pm$0.006   
                         DCT          0.934$\pm$0.01  0.941$\pm$0.006   
             sensitivity SVM         0.951$\pm$0.006  0.964$\pm$0.004   
                         LR          0.939$\pm$0.012  0.954$\pm$0.007   
                         DCT         0.931$\pm$0.013   0.94$\pm$0.008   
20%          F-1 score   SVM         0.959$\pm$0.009  0.967$\pm$0.011   
                         LR          0.939$\pm$0.006  0.954$\pm$0.012   
                         DCT         0.943$\pm$0.016  0.939$\pm$0.012   
             sensitivity SVM         0.957$\pm$0.009   0.97$\pm$0.008   
                         LR          0.927$\pm$0.007   0.95$\pm$0.012   
                         DCT          0.936$\pm$0.02  0.935$\pm$0.011   
30%          F-1 score   SVM          0.959$\pm$0.01   0.962$\pm$0.01   
                         LR          0.942$\pm$0.011  0.958$\pm$0.009   
                         DCT          0.939$\pm$0.01   0.94$\pm$0.013   
             sensitivity SVM

In [13]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_oil-low.csv", index = False)

In [14]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
0\% & F-1 score & SVM &  0.491\$\textbackslash pm\$0.053 &   0.959\$\textbackslash pm\$0.01 &  0.957\$\textbackslash pm\$0.009 &   0.96\$\textbackslash pm\$0.007 &  0.948\$\textbackslash pm\$0.008 &   0.961\$\textbackslash pm\$0.01 \\
    &             & LR &  0.539\$\textbackslash pm\$0.081 &  0.951\$\textbackslash pm\$0.009 &   0.945\$\textbackslash pm\$0.01 &  0.955\$\textbackslash pm\$0.009 &  0.914\$\textbackslash pm\$0.009 &   0.95\$\textbackslash pm\$0.007 \\
    &             & DCT &  0.512\$\textbackslash pm\$0.138 &  0.946\$\textbackslash pm\$0.009 &  0.942\$\textbackslash pm\$0.007 &   0.941\$\textbackslash pm\$0.01 &   0.87\$\textbackslash pm